### 이미지 분류 CNN 만들기 목차

* [Chapter 1 AlexNet](#chapter1)
* [Chapter 2 VGGnet](#chapter2)
* [Chapter 3 합성곱층 만들기](#chapter3)
   * [Section 3.1 입력과 출력사이의 관계](#section_3_1)
   * [Section 3.2 풀링층과 밀집층](#section_3_2)

### Chapter 1 AlexNet <a class="anchor" id="chapter1"></a>
1. 기존의 이미지 처리 방법들은 이미지 분류의 정확도를 높이기 위해 수동으로 여러 특성을만듬

2. AlexNet은 이미지 원본 픽셀을 그대로 사용하면서 다른 모델과 비교해 성능면에서 압도적인 차이를 보여줌

3. AlexNet 모델은 LetNet-5 모델보다 많은 층을 사용한다.

   ![AlexNet](image/02-01-AlexNet2.png)    

4. AlexNet 모델은 LetNet-5에서 사용한 활성화 함수인 시그모이드 대신 렐루 함수를 사용한다.
   - 여러 개의 층을 가진 신경만의 경우, 시작 부분의 층을 업데이트 하려면 뒤쪽 층에 있는 기울기 부터 누적해야한다.
   - 기울기가 작다면 누적된 기울기도 작어져서 파라미터를 업데이트 하는데 거의 영향을 미치지 못하는 기울기 소실 문제가 발생한다.
   - AlexNet은 렐루 함수를 사용하여 기울기 소실문제를 개선하였다.

5. 평균 풀링대신 최대 풀링을 사용한다.
   - 일반적인 풀링은 스트라이 크기와 플링 크기가 같지만, AlexNet은 스트라이드 크기가 풀링 크기모다 작어 중첩된 풀링을 진행한다.
   - 중요한 특징 정보를 희석시키지 않는다.

6. AlexNet 모델은 밀집응의 과대적합을 막기 위해 유닛의 출력을 랜덤하게 끄는 드롭아웃을 사용한다.

7. 이미지 넷에 있는 이미지의 크기는 가장 작은 크기가 256픽셀이고, 최대 512 픽셀을 넘지 않는다.
   - AlexNet은 227*227의 크기의 이미지를 사용, 컬러 이미지를 사용하므로 입력 채널의 깊이는 3이된다.
   - Input()를 통해 입력의 크기지정 -> 2개의 합성곱층과 풀링층 -> 3개의 합성곱층 -> 1개의 플링층 -> 2개의 밀집층과 드롭아웃 -> 출력

      ![AlexNet](image/02-01-AlexNet3.png)    
      

In [1]:
import keras
from keras import layers

alexnet = keras.Sequential()

# 입력(컬러 이미지)의 크기지정
alexnet.add(layers.Input(shape=(227, 227, 3)))

# 1번째 합성곱층
alexnet.add(layers.Conv2D(filters=96, kernel_size=11, strides=4, activation='relu'))

# 1번째 풀링층
alexnet.add(layers.MaxPooling2D(pool_size=3, strides=2))

# 2번째 합성곱층 - same 패딩을 사용하여 입력과 특성 맵의 크기를 동일하게 유지
alexnet.add(layers.Conv2D(filters=256, kernel_size=5, padding='same', activation='relu'))

# 2번째 풀링층
alexnet.add(layers.MaxPooling2D(pool_size=3, strides=2))

# 3번째 합성곱층 - same 패딩을 사용하여 입력과 특성 맵의 크기를 동일하게 유지
alexnet.add(layers.Conv2D(filters=384, kernel_size=3, padding='same', activation='relu'))   

# 4번째 합성곱층 - same 패딩을 사용하여 입력과 특성 맵의 크기를 동일하게 유지
alexnet.add(layers.Conv2D(filters=384, kernel_size=3, padding='same', activation='relu'))  

# 5번째 합성곱층 - same 패딩을 사용하여 입력과 특성 맵의 크기를 동일하게 유지
alexnet.add(layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'))

# 3번째 풀링층
alexnet.add(layers.MaxPooling2D(pool_size=3, strides=2))

# 완전 연결층에 전달하기 위해 1차원 배열로 변환
alexnet.add(layers.Flatten())

# 1번째 완전 연결층
alexnet.add(layers.Dense(4096, activation='relu'))  

# 드롭아웃 적용
alexnet.add(layers.Dropout(0.5))  

# 2번째 완전 연결층
alexnet.add(layers.Dense(4096, activation='relu'))

# 드롭아웃 적용
alexnet.add(layers.Dropout(0.5))

# 출력층 - 1000개의 클래스에 대한 소프트맥스 활성화 함수 사용
alexnet.add(layers.Dense(1000, activation='softmax'))

alexnet.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 55, 55, 96)     │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 27, 27, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 27, 27, 256)    │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 13, 13, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 13, 13, 384)    │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 13, 13, 384)    │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 13, 13, 256)    │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │    37,752,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1000)           │     4,097,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 62,378,344 (237.95 MB)

 Trainable params: 62,378,344 (237.95 MB)

 Non-trainable params: 0 (0.00 B)

### Chapter 2 VGGNet <a class="anchor" id="chapter2"></a>
1. VGGNet(브이지지넷)은 옥스포드 대학교에서 만든 합성곱 신경망구조로 2014 이미지 넷에서 준우승을한 모델이다.

2. 합성곱층과 풀링층을 교대로 반복하는 대신, 여러번의 합성곱층을 적용한 다음 풀링층을 적용한다.
   - 높은 성능을 내는 깊은 네트워크를 만들기 위해 풀링을 적용하기 전에 여러개의 합성곱층을 배치

3. 동일한 여러개의 합성곱층과 풀링층을 반복하는 블록(block)를 적용한다.
   - 블록은 간단하지만 폴은 성능을 달성하여 이후 많은 신경망고주에 영향을 주었다.

4. 큰 필터를 사용하는 하나의 합성곱 대신, 3*3 크기의 작은 필터를 사용하는 여러 개의 합성곱층을 적용

    ![VGGNet](image/02-01-VGGNet2.png)    



In [3]:
import keras
from keras import layers

vggnet = keras.Sequential()

# 입력(컬러 이미지)의 크기지정
vggnet.add(layers.Input(shape=(224, 224, 3)))

# 1,2 번째 블록 : 64개의 합성곱 필터와 128개의 합성곱 필트럴 사용하는 블록 
for n_filters in [64, 128]:
    # 2개의 합성곱층과 1개의 풀링층 추가
    for _ in range(2):
        vggnet.add(layers.Conv2D(filters=n_filters, kernel_size=3, padding='same', activation='relu'))
    vggnet.add(layers.MaxPooling2D(pool_size=2, strides=2))

# 3,4,5 번째 블록 : 256개의 합성곱 필터와 512개의 합성곱 필트럴 사용하는 블록    
for n_filters in [256, 512, 512]:
    # 3개의 합성곱층과 1개의 풀링층 추가
    for _ in range(3):
        vggnet.add(layers.Conv2D(filters=n_filters, kernel_size=3, padding='same', activation='relu'))
    vggnet.add(layers.MaxPooling2D(pool_size=2, strides=2))
    
# 완전 연결층에 전달하기 위해 1차원 배열로 변환
vggnet.add(layers.Flatten())

# 1번째 완전 연결층
vggnet.add(layers.Dense(4096, activation='relu'))

# 2번째 완전 연결층
vggnet.add(layers.Dense(4096, activation='relu'))

# 출력층 - 1000개의 클래스에 대한 소프트맥스 활성화 함수 사용
vggnet.add(layers.Dense(1000, activation='softmax'))

vggnet.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1000)           │     4,097,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 138,357,544 (527.79 MB)

 Trainable params: 138,357,544 (527.79 MB)

 Non-trainable params: 0 (0.00 B)